<a href="https://colab.research.google.com/github/kangning-huang/PopWeightedUHI/blob/main/Population_weighted_SUHI_extremes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# download geemap (https://geemap.org/)
!pip install geemap &> /dev/null
# download geopandas (https://geopandas.org/)
!pip install geopandas &> /dev/null

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import geopandas as gdp
import geemap
from geemap import geojson_to_ee, ee_to_geojson

In [13]:
# A Google Earth Engine account is required to authenticate geemap.
# Register here https://earthengine.google.com/
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wirkyxHqf6CJdAXQHDgVGqGXAAcElrdv4QmjRKiNjGc&tc=_arJKc-C6tz9gtevbvNQaNDP47fabftA3igWw3ZEN9I&cc=clTr8JvXiY5dlO6GlMUhsqsuWfx2h4NZWYNglFMQQFI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUU0nPfG-e5zF5gpJiYv1vfFMBlkBnVRFCWTHRPAS98c2AFJfqKfyw

Successfully saved authorization token.


In [14]:
# Download the functional urban areas from Gobal Human Settlement Layer.
!curl https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL//GHS_FUA_UCDB2015_GLOBE_R2019A/V1-0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.zip --output GHS_FUA.zip
!unzip GHS_FUA.zip
gdf_FUAs = gdp.read_file('/content/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')
gdf_FUAs_sel  = gdf_FUAs.sort_values('FUA_p_2015', ascending=False).head(1100)
# gdf_FUAs_sel.head()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2987k  100 2987k    0     0  2246k      0  0:00:01  0:00:01 --:--:-- 2246k
Archive:  GHS_FUA.zip
replace GHSL_FUA_2019.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: GHSL_FUA_2019.pdf       
  inflating: GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg  


In [17]:
# Function to get UHI stats by year
def GetUhiStatsByYear(year):
  # getting the image collection
  uhiModis1 = ee.ImageCollection('users/lorenzomentaschi/uhiModis');
  uhiModis2 = ee.ImageCollection('users/lucfeyen3/uhiModis');
  uhiModis = uhiModis1.merge(uhiModis2);
  # uhiModis3 = ee.ImageCollection('users/lucfeyen3/uhiModisStats_Shanghai');
  # uhiModis = uhiModis1.merge(uhiModis2).merge(uhiModis3);
  # Filter images in the year
  filterYear = ee.Filter.calendarRange(year, year, 'year')
  # Filter pixles with more than 50 rual pixels
  def FilterByMinNRuralPixel(img):
    minNRuralPixel = 50
    ruralPxlCnt = img.select('N_NOTURB_PIX_Day')
    msk = ruralPxlCnt.gte(minNRuralPixel)
    return img.select(['UHI_Day_1km', 'UHI_Night_1km']).updateMask(msk)
  # Apply both filters
  ImColYear = ee.ImageCollection(uhiModis.filter(filterYear).map(FilterByMinNRuralPixel))
  date = ImColYear.first().get('system:time_start')
  ImStatsYear = ImColYear.reduce(ee.Reducer.percentile([99]))
  return ImStatsYear.set({'system:time_start': date})

# Function to get Landscan population by year
def GetLandscanPopByYear(year):
  ImColPop = ee.ImageCollection("projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL")
  ImPopYear = ImColPop.filter(ee.Filter.calendarRange(year,year,'year')).first()
  return ImPopYear

# Function to get POP times UHI
def GetUhiStatsTimesPopByYear(year):
  ImUhiStat = GetUhiStatsByYear(year)
  ImPop = GetLandscanPopByYear(year)
  return ImUhiStat.multiply(ImPop)

# Get global warm season mean from an image collection
def GetWarmSeasonMean(imgCol, year):
  # Filter by year
  imgCol = imgCol.filter(ee.Filter.calendarRange(year, year, 'year'))
  # Masks for Nothern (NH) and Southern (SN) Hemispheres
  maskNH = ee.Image.pixelLonLat().select('latitude').gt(0)
  maskSH = ee.Image.pixelLonLat().select('latitude').lt(0);
  # Masks for climate zones
  maskTemperate = ee.Image.pixelLonLat().select('latitude').abs().gt(35)
  maskSubtropic = ee.Image.pixelLonLat().select('latitude').abs().lt(35).And(
                  ee.Image.pixelLonLat().select('latitude').abs().gt(10))
  maskTropical  = ee.Image.pixelLonLat().select('latitude').abs().lt(10)
  # Filters for months in warm seasons
  filterJJA = ee.Filter.calendarRange(6, 8, 'month')
  filterDJF = ee.Filter.Or(
      ee.Filter.calendarRange( 1,  2, 'month'),
      ee.Filter.calendarRange(12, 12, 'month'))
  filterAMJJAS = ee.Filter.calendarRange(4, 9, 'month')
  filterONDJFM = ee.Filter.Or(
      ee.Filter.calendarRange(10, 12, 'month'),
      ee.Filter.calendarRange( 1,  3, 'month'))
  # Filter images of warm seasons in different (climates + hemispheres)
  imgColTemperateN = imgCol.filter(filterJJA)
  imgColTemperateS = imgCol.filter(filterDJF)
  imgColSubtropicN = imgCol.filter(filterAMJJAS)
  imgColSubtropicS = imgCol.filter(filterONDJFM)
  # Calculate the masked means
  imgMeanTemperateN = imgColTemperateN.mean().updateMask(maskTemperate.And(maskNH))
  imgMeanTemperateS = imgColTemperateS.mean().updateMask(maskTemperate.And(maskSH))
  imgMeanSubtropicN = imgColSubtropicN.mean().updateMask(maskSubtropic.And(maskNH))
  imgMeanSubtropicS = imgColSubtropicS.mean().updateMask(maskSubtropic.And(maskSH))
  imgMeanTropical   = imgCol.mean().updateMask(maskTropical)
  # Return the mosaiced image
  return ee.ImageCollection.fromImages([imgMeanTemperateN, imgMeanTemperateS,
                                        imgMeanSubtropicN, imgMeanSubtropicS,
                                        imgMeanTropical]).mosaic()



# Define function to calculate UHI per cap for FUA, in a year
def GetUhiPercapFUAbyYear(gdf_fua, year, urbanMasked = False):
  fc_fua = geemap.geopandas_to_ee(gdf_fua).first()
  geo_fua = fc_fua.geometry()
  uhi_cnt = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.count(), geo_fua, 1000).getInfo()
  if uhi_cnt['UHI_Day_1km_p99'] > 0:
    # ID and name of the FUA
    fua_id   = gdf_fua[['eFUA_ID']].iloc[0].iloc[0]
    fua_name = gdf_fua[['eFUA_name']].iloc[0].iloc[0]
    # Get Landscan population image for the year
    ImPopYear = GetLandscanPopByYear(year)
    # Area-area SUHI
    uhi_mean = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.mean(), geo_fua, 1000).getInfo()
    # Total population
    pop_sum_1km = ImPopYear.reduceRegion(ee.Reducer.sum(), geo_fua, 1000).getInfo()
    # Total SUHI times population
    uhiTimesPop_sum = GetUhiStatsTimesPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 1000).getInfo()
    # Population-weighted SUHI
    uhi_day_p99_percap = uhiTimesPop_sum['UHI_Day_1km_p99'] / pop_sum_1km['b1']
    # Calculate population density times population
    ImPopTimesPopDensYear = ImPopYear.multiply(ImPopYear)
    # Calculate total population for FUA
    pop_total = ImPopYear.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=fc_fua.geometry(),
        scale=1000, maxPixels=1e13).getInfo()['b1']
    # Total pop-density times population
    pop_dens_times_pop_total = ImPopTimesPopDensYear.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=fc_fua.geometry(),
        scale=1000, maxPixels=1e13).getInfo()['b1']
    # Calculate the population density per capita for FUA
    pop_dens_percap = pop_dens_times_pop_total / pop_total
    # Get warm season Mean Albedo of the year (https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD43A3#bands)
    ImAlbYear = GetWarmSeasonMean(ee.ImageCollection("MODIS/061/MCD43A3"), year).select('Albedo_WSA_shortwave').multiply(0.001)
    # Get warm season Mean EVI of the year (https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A2#bands)
    ImEviYear = GetWarmSeasonMean(ee.ImageCollection("MODIS/061/MOD13A2"), year).select('EVI').multiply(0.0001)
    # Calculate population times Albedo
    ImPopTimesAlbYear = ImPopYear.multiply(ImAlbYear)
    # Calculate population times EVI
    ImPopTimesEviYear = ImPopYear.multiply(ImEviYear)
    # Calculate the total Albedo times population for FUA
    alb_times_pop_total = ImPopTimesAlbYear.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=fc_fua.geometry(),
        scale=1000, maxPixels=1e13).getInfo()['b1']
    # Calculate the total EVI times population for FUA
    evi_times_pop_total = ImPopTimesEviYear.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=fc_fua.geometry(),
        scale=1000, maxPixels=1e13).getInfo()['b1']
    # Calculate the albedo per capita for FUA
    alb_percap = alb_times_pop_total / pop_total
    # Calculate the EVI per capita for FUA
    evi_percap = evi_times_pop_total / pop_total
    # Construct a new data frame
    df_new = pd.DataFrame({
        'eFUA_ID': fua_id,
        'eFUA_name': fua_name,
        'Year': year,
        'pop_total': int(pop_total),
        'SUHI_A': [uhi_mean['UHI_Day_1km_p99']],
        'SUHI_P': [uhi_day_p99_percap],
        'pop_dens': int(pop_dens_percap),
        'albedo': alb_percap,
        'EVI': evi_percap
    }, index=[str(fua_id) + '_' + str(year)])
  # Return the population density per capita for FUA
  return df_new

In [ ]:
filename_out = 'pop_weighted_SUHI.csv'

# Create empty data frames for results
df_pop_w_SUHI = pd.DataFrame()

# Test 10 FUAs
gdf_FUAs_sel  = gdf_FUAs.sort_values('FUA_p_2015', ascending=False).head(10)

# Loop through FUAs
for fua_id in tqdm(gdf_FUAs_sel['eFUA_ID'], total=gdf_FUAs_sel.shape[0]):
  for year in range(2003, 2021):
    gdf_fua = gdf_FUAs.loc[gdf_FUAs['eFUA_ID']==fua_id]
    # Add to the result data frame
    df_pop_w_SUHI_new = GetUhiPercapFUAbyYear(gdf_fua, year)
    if df_pop_w_SUHI_new.empty==False:
      df_pop_w_SUHI = pd.concat([df_pop_w_SUHI, df_pop_w_SUHI_new])
      df_pop_w_SUHI.to_csv(filename_out)

 50%|█████     | 5/10 [17:28<15:01, 180.30s/it]